In [1]:
import os
from getpass import getpass

GOOGLE_API_KEY = getpass('Enter Gemini Key : ')
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY

Enter Gemini Key : ··········


In [2]:
!pip install -q llama-index-core==0.10.30 #The core llamaindex package
!pip install -q llama-index-llms-gemini==0.1.7 #The package for Google Gemini LLM
!pip install -q llama-index-embeddings-gemini==0.1.6 #The package for using Gemini embeddings
!pip install -q llama-index-readers-file==0.1.19 #For reading files

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.4/137.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 598.7/598.7 kB 24.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. Thi

In [3]:
!mkdir data
!wget --user-agent "Mozilla" "https://arxiv.org/pdf/2307.09288.pdf" -O ./data/llama_2.pdf #Downloading Llama 2 paper

--2024-06-04 08:25:32--  https://arxiv.org/pdf/2307.09288.pdf
Resolving arxiv.org (arxiv.org)... 151.101.3.42, 151.101.67.42, 151.101.195.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.3.42|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://arxiv.org/pdf/2307.09288 [following]
--2024-06-04 08:25:32--  http://arxiv.org/pdf/2307.09288
Connecting to arxiv.org (arxiv.org)|151.101.3.42|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13661300 (13M) [application/pdf]
Saving to: ‘./data/llama_2.pdf’

./data/llama_2.pdf  100%[===================>]  13.03M  --.-KB/s    in 0.08s   

2024-06-04 08:25:32 (172 MB/s) - ‘./data/llama_2.pdf’ saved [13661300/13661300]



## Popular node parsers

In [4]:
from llama_index.core.settings import Settings
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.llms.gemini import Gemini

Settings.llm = Gemini(model_name="models/gemini-pro", temperature = 0.0)
Settings.embed_model = GeminiEmbedding(model_name="models/embedding-001")

In [9]:
from llama_index.readers.file import PDFReader

pdf_reader = PDFReader(return_full_document = True)
doc = pdf_reader.load_data(file='./data/llama_2.pdf')

In [10]:
help(doc)

Help on list object:

class list(object)
 |  list(iterable=(), /)
 |  
 |  Built-in mutable sequence.
 |  
 |  If no argument is given, the constructor creates a new empty list.
 |  The argument must be an iterable if specified.
 |  
 |  Methods defined here:
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __contains__(self, key, /)
 |      Return key in self.
 |  
 |  __delitem__(self, key, /)
 |      Delete self[key].
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getitem__(...)
 |      x.__getitem__(y) <==> x[y]
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __iadd__(self, value, /)
 |      Implement self+=value.
 |  
 |  __imul__(self, value, /)
 |      Implement self*=value.
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate sign

In [11]:
len(doc) #There is only one doc.
#We cannot process the entire doc and hence we have to chunk the document into nodes.

1

In [12]:
from llama_index.core.node_parser import (
    SentenceSplitter,
    SentenceWindowNodeParser,
    SemanticSplitterNodeParser
)

In [13]:
sentence_splitter = SentenceSplitter(
    chunk_size = 256,
    chunk_overlap = 16
)
#Use this if thorough understanding of the input is there
nodes1 = sentence_splitter.get_nodes_from_documents(doc, show_progress = True)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

In [14]:
len(nodes1)

336

In [22]:
nodes1[0].relationships

{<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='d389f536-6436-416e-9566-c8982647f7c1', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_name': 'llama_2.pdf'}, hash='678fe8e87c74dae38dc3711362072763088517150208c87778e67fba376252b2'),
 <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='ac7ada80-5723-4316-a307-21565988bfb6', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='1b546e71ee908eaf5e3a05bf11b13a8e75d32f3989f9849c6eb01e43dd3743b3')}

In [16]:
nodes1[0].get_text()

'Llama 2 : Open Foundation and Fine-Tuned Chat Models\nHugo Touvron∗Louis Martin†Kevin Stone†\nPeter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra\nPrajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen\nGuillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller\nCynthia Gao Vedanuj Goswami Naman Goyal Anthony Hartshorn Saghar Hosseini Rui Hou\nHakan Inan Marcin Kardas Viktor Kerkez Madian Khabsa Isabel Kloumann Artem Korenev\nPunit Singh Koura Marie-Anne Lachaux Thibaut Lavril Jenya Lee Diana Liskovich\nYinghai Lu Yuning Mao Xavier Martinet Todor Mihaylov Pushkar Mishra\nIgor Molybog Yixin Nie Andrew Poulton Jeremy Reizenstein Rashi Rungta Kalyan Saladi\nAlan Schelten Ruan Silva Eric Michael Smith Ranjan Subramanian Xiaoqing Ellen Tan'

In [17]:
sentence_window_parser = SentenceWindowNodeParser(
    window_size = 3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

nodes2 = sentence_splitter.get_nodes_from_documents(doc, show_progress = True)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

In [27]:
nodes2[0].get_text()
# Use this when you are trying to find fine grained answers within a sentence.
#The surrounding sentences with a window size is added as metadata along with the original text.
# This is used in conjunction with the "MetadataReplacementPostProcessor" for better results.

'Llama 2 : Open Foundation and Fine-Tuned Chat Models\nHugo Touvron∗Louis Martin†Kevin Stone†\nPeter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra\nPrajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen\nGuillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller\nCynthia Gao Vedanuj Goswami Naman Goyal Anthony Hartshorn Saghar Hosseini Rui Hou\nHakan Inan Marcin Kardas Viktor Kerkez Madian Khabsa Isabel Kloumann Artem Korenev\nPunit Singh Koura Marie-Anne Lachaux Thibaut Lavril Jenya Lee Diana Liskovich\nYinghai Lu Yuning Mao Xavier Martinet Todor Mihaylov Pushkar Mishra\nIgor Molybog Yixin Nie Andrew Poulton Jeremy Reizenstein Rashi Rungta Kalyan Saladi\nAlan Schelten Ruan Silva Eric Michael Smith Ranjan Subramanian Xiaoqing Ellen Tan'

In [23]:
nodes2[1].relationships # We have the previous node and the next node.
#This automatically creates relationships among the nodes.

{<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='d389f536-6436-416e-9566-c8982647f7c1', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_name': 'llama_2.pdf'}, hash='678fe8e87c74dae38dc3711362072763088517150208c87778e67fba376252b2'),
 <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='28209db4-ec8f-492b-889e-fbbf7fd343c3', node_type=<ObjectType.TEXT: '1'>, metadata={'file_name': 'llama_2.pdf'}, hash='092119fc23c530a173ae31134e8cbcc06519e0e7ed48a7ef6c3f980970f664de'),
 <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='6ddd5847-84a0-4ce3-9bc6-b51e49f3a3a3', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='556da8799084cfa9b2ae93fcbe40051668e38ddeee4b57bb9cd469f7e19e0e28')}

<b> Algorithm for semantic chunking</b>

<pre>
1. Generate embeddings for every sentence.
2. Using sentence similarity start grouping sentences from left to right.
    a. Start with sentence 1. Generate embedding.
    b. Generate the embedding for sentence 2. Compare the sentence similarity with sentence 1.
    c. Optional to generate combined embeddings of sentence 1 and 2 to be compared with sentence 3.
    d. Have a threshold defined below which the current grouping stops.
    
</pre>

In [25]:
semantic_chunker = SemanticSplitterNodeParser(
    buffer_size = 1, #Window size of grouping point c
    breakpoint_percentile_threshold = 95,
    embed_model = GeminiEmbedding(model_name="models/embedding-001")
)

In [26]:
nodes3 = semantic_chunker.get_nodes_from_documents(doc, show_progress = True)
# This is expensive but should group nodes with semantic meanings together.
# We can also define the sentence splitter logic.

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2889 [00:00<?, ?it/s]

KeyboardInterrupt: 

## Metadata extractors

In [28]:
# Let us generate metadata for a sample of 2 nodes from sentence splitter

meta_nodes = nodes1[:2]

In [29]:
llm = Gemini(model_name="models/gemini-pro", temperature = 0.0)

In [31]:
# We shall use ingestion pipeline to extract the following metadata
# 1. Questions that this node can answer.
# 2. Summarization of the node.
# 3. A suitable title for a set of nodes.

from llama_index.core.extractors import SummaryExtractor, QuestionsAnsweredExtractor, TitleExtractor
from llama_index.core.schema import MetadataMode


extractors = [
    SummaryExtractor(summaries=["prev", "self", "next"], llm = llm), #The previous node, the current node and the next node
    QuestionsAnsweredExtractor(questions = 3, metadata_mode = MetadataMode.EMBED, llm = llm), #Include metadata for embedding
    TitleExtractor(nodes = 3), #number of nodes from front to use for title extraction
]

In [32]:
import nest_asyncio

#Applicable only for Jupyter notebooks
nest_asyncio.apply()

In [33]:
from llama_index.core.ingestion import IngestionPipeline

pipeline = IngestionPipeline(
    name = 'Metadata Extractor',
    transformations = [*extractors]
)
meta_sample_nodes = pipeline.run(show_progress = True, nodes = meta_nodes)

100%|██████████| 2/2 [00:04<00:00,  2.28s/it]


In [34]:
meta_sample_nodes[0].metadata.keys()

dict_keys(['file_name', 'next_section_summary', 'section_summary', 'questions_this_excerpt_can_answer', 'document_title'])

In [35]:
meta_sample_nodes[0].metadata

{'file_name': 'llama_2.pdf',
 'next_section_summary': '**Key Topics:**\n\n* Release of Llama 2, a collection of pretrained and fine-tuned large language models (LLMs)\n* Fine-tuned LLMs (Llama 2-Chat) optimized for dialogue use cases\n* Comparison of Llama 2-Chat to open-source chat models\n* Humane evaluations of Llama 2-Chat for helpfulness and safety\n* Description of fine-tuning and safety improvements for Llama 2-Chat\n\n**Entities:**\n\n* Llama 2\n* Llama 2-Chat\n* GenAI\n* Meta\n* Authors: Ruan Silva, Eric Michael Smith, Ranjan Subramanian, Xiaoqing Ellen Tan, Binh Tang, Ross Taylor, Adina Williams, Jian Xiang Kuan, Puxin Xu, Zheng Yan, Iliyan Zarov, Yuchen Zhang, Angela Fan, Melanie Kambadur, Sharan Narang, Aurelien Rodriguez, Robert Stojnic, Sergey Edunov, Thomas Scialom',
 'section_summary': '**Key Topics:**\n\n* Open Foundation and Fine-Tuned Chat Models\n\n**Entities:**\n\n* **Authors:**\n    * Hugo Touvron\n    * Louis Martin\n    * Kevin Stone\n    * Peter Albert Amjad Al